<h2> Data </h2>

In [18]:
import pandas as pd

# Create dataFrame from out data file (csv)
df = pd.read_csv("./nfl-scores-and-betting-data/spreadspoke_scores.csv")
# Drop columns with information we don't need
df = df.drop(columns=["schedule_season", "schedule_week", "schedule_playoff", "stadium_neutral", "team_favorite_id", "spread_favorite", "over_under_line"],axis=1)
# Drop games where stadium was closed
df = df[df.weather_detail != "DOME"]
# Print out dataFrame
df

,schedule_date,team_home,score_home,score_away,team_away,stadium,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,09/02/1966,Miami Dolphins,14,23,Oakland Raiders,Orange Bowl,83.0,6.0,71,NaN
1,09/03/1966,Houston Oilers,45,7,Denver Broncos,Rice Stadium,81.0,7.0,70,NaN
2,09/04/1966,San Diego Chargers,27,7,Buffalo Bills,Balboa Stadium,70.0,7.0,82,NaN
3,09/09/1966,Miami Dolphins,14,19,New York Jets,Orange Bowl,82.0,11.0,78,NaN
4,09/10/1966,Green Bay Packers,24,3,Baltimore Colts,Lambeau Field,64.0,8.0,62,NaN
5,09/10/1966,Houston Oilers,31,0,Oakland Raiders,Rice Stadium,77.0,6.0,82,NaN
6,09/10/1966,San Diego Chargers,24,0,New England Patriots,Balboa Stadium,69.0,9.0,81,NaN
7,09/11/1966,Atlanta Falcons,14,19,Los Angeles Rams,Atlanta-Fulton County Stadium,71.0,7.0,57,NaN
8,09/11/1966,Buffalo Bills,20,42,Kansas City Chiefs,War Memorial Stadium,63.0,11.0,73,NaN
9,09/11/1966,Detroit Lions,14,3,Chicago Bears,Tiger Stadium,67.0,7.0,73,NaN
